In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# Initialize Chrome WebDriver with the specified path and options
service = Service(PATH)
# chrome = webdriver.Chrome(service=service, options=chrome_options)

# player_ids = ['677950', '572233', '682998', '666150', '656896', '553993', '672515', '672695', '607054', '664983', '645444', '606466', '666971', '656976', '592626']
timeout = 25
mlb_seasons = ['2023', '2022', '2021']
player_ids = ['677950']

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [4]:
url = r"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId=502671&type=&batHand=&pitchHand=&pitchType=&year=2023&zone="
driver.get(url)

element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
WebDriverWait(driver, timeout).until(element_present)

headers_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations"]/table/thead/tr'))
WebDriverWait(driver, timeout).until(headers_present)

select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
select.select_by_index(0)
pitchType = select.first_selected_option.text
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "calculations"})
headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist[0] = "Name"

stats = pd.DataFrame(columns=headerlist)

In [5]:
for player in player_ids:
    for season in mlb_seasons:
        for i in range(len(select.options)):
            url = f"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId={player}&type=&batHand=&pitchHand=&pitchType=&year={season}&zone="
            driver.get(url)

            element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
            WebDriverWait(driver, timeout).until(element_present)

            data_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations-tr_0"]'))
            WebDriverWait(driver, timeout).until(data_present)
            
            select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
            select.select_by_index(f"{i}")

            pitches_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="viz"]/div[2]/h6/h6'))
            WebDriverWait(driver, timeout).until(pitches_present)
            pitches_message = driver.find_element(By.XPATH, r'//*[@id="viz"]/div[2]/h6/h6').text
            if pitches_message == 'No Data Found':
                break
                
            pitchType = select.first_selected_option.text
            src = driver.page_source
            parser = BeautifulSoup(src, "lxml")
            table = parser.find("div", attrs = {"id": "calculations"})
            headers = table.findAll('th')
            headerlist = [h.text.strip() for h in headers[:]]
            headerlist[0] = "Name"
            rows = table.findAll('tr')[1:]
            player_stats = [[td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]
            if player_stats[0][4] == "": player_stats[0][4] = pitchType
            if player_stats[0][1] == "": player_stats[0][1] = season

            stats2 = pd.DataFrame(player_stats, columns=headerlist)
            stats = pd.concat([stats,stats2], ignore_index=True)

driver.quit()

In [6]:
stats

,Name,Season,Total Pitches,Pitches,Pitch Types,%,PA,AB,H,1B,...,BA,SO,Swings,Misses,Sw%,Whiff%,K%,wOBA,EV,Spin Rate
0,Thomas,2023,1463,1463,All Pitches,100.0,394,368,85,54,...,.231,83,720,178,49.2,24.7,21.1,.281,88.2,2249
1,Thomas,2023,1463,745,Fastballs,50.9,195,180,48,27,...,.267,31,363,78,48.7,21.5,15.9,.338,89.9,2253
2,Thomas,2023,1463,236,Offspeed,16.1,73,69,13,9,...,.188,15,118,28,50.0,23.7,20.5,.211,85.4,1686
3,Thomas,2023,1463,482,Breaking,32.9,126,119,24,18,...,.202,37,239,72,49.6,30.1,29.4,.233,87.0,2517
4,Thomas,2023,1463,193,Changeup,13.2,59,55,10,8,...,.182,12,95,24,49.2,25.3,20.3,.191,84.0,1754
5,Thomas,2023,1463,174,Curveball,11.9,41,39,3,3,...,.077,13,82,25,47.1,30.5,31.7,.099,87.1,2623
6,Thomas,2023,1463,0,Eephus,0.0,0,0,0,0,...,--,0,0,0,--,--,--,--,--,--
7,Thomas,2023,1463,0,BP Fastball,0.0,0,0,0,0,...,--,0,0,0,--,--,--,--,--,--
8,Thomas,2023,1463,116,Cutter,7.9,33,31,10,4,...,.323,2,62,13,53.4,21.0,6.1,.454,87.3,2358
9,Thomas,2023,1463,438,4-Seam Fastball,29.9,108,96,29,17,...,.302,18,213,47,48.6,22.1,16.7,.372,90.5,2278
